In [1]:
# Thanks to tsogtochir.e's public code release (https://www.kaggle.com/code/easterndundrey/csiro-gold-solution), this post was possible.

import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import random
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
# ======== Load Datafile ========
test_df = pd.read_csv('/kaggle/input/csiro-biomass/test.csv')
test_df["id"] = test_df["sample_id"].str.split("__").str[0]
image_paths_df = test_df[["id","image_path"]].drop_duplicates("image_path").reset_index(drop=True)

In [3]:
# ======== Seed ========
def seed_everything(seed: int = 114514):
    pl.seed_everything(seed, workers=True)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # 
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(114514)

In [4]:
# ======== Dataset ========
class InferenceDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open('/kaggle/input/csiro-biomass/' + row["image_path"]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

In [5]:
# ======== Loss Funtion ========
class WeightedR2Loss(nn.Module):
    def __init__(self, weights=None, eps=1e-8):
        super().__init__()
        if weights is None:
            weights = torch.tensor([0.1, 0.1, 0.1, 0.2, 0.5])
        self.register_buffer('weights', weights)
        self.eps = eps

    def forward(self, y_pred, y_true):
        """
        y_pred: (B, 3)
        y_true: (B, 5)
        """
        DG = y_pred[:,0]
        GDM = y_pred[:,1]
        DT = y_pred[:,2]
        DD = DT - GDM
        DC = GDM - DG
        y_hat = torch.stack([DG, DD, DC, GDM, DT], dim=1)
        
        y_mean = torch.mean(y_true, dim=0, keepdim=True)
        ss_res = torch.sum((y_true - y_hat) ** 2, dim=0)
        ss_tot = torch.sum((y_true - y_mean) ** 2, dim=0)
        r2 = 1 - ss_res / (ss_tot + self.eps)

        weighted_r2 = torch.sum(self.weights * r2)
        loss = 1 - weighted_r2

        return loss

In [6]:
# ======== Model ========
class MultiRegressionModel(pl.LightningModule):
    def __init__(
        self,
        model_name="efficientnet_b2",
        pretrained=False,
        lr=1e-4,
        output_dim=3,
    ):
        super().__init__()
        self.save_hyperparameters()
        self.backbone = timm.create_model(
            model_name, 
            pretrained=pretrained, 
            num_classes=output_dim, 
            global_pool="avg"
        )

        self.criterion = WeightedR2Loss()
        self.val_outputs = []

    def forward(self, x_img):
        return self.backbone(x_img)

    def training_step(self, batch, batch_idx):
        x_img, y = batch
        y_hat = self(x_img)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x_img, y = batch
        y_hat = self(x_img)
        loss = self.criterion(y_hat, y)
        self.val_outputs.append((y_hat.detach().cpu(), y.detach().cpu()))
        self.log("val_loss", loss, on_step=False, on_epoch=True)
        return loss

    def on_validation_epoch_end(self):
        y_hats = torch.cat([x[0] for x in self.val_outputs], dim=0)
        y_trues = torch.cat([x[1] for x in self.val_outputs], dim=0)
        self.val_outputs.clear()

        DG = y_hats[:, 0]
        GDM = y_hats[:, 1]
        DT = y_hats[:, 2]
        DD = DT - GDM
        DC = GDM - DG
        y_hat_full = torch.stack([DG, DD, DC, GDM, DT], dim=1)

        y_mean = torch.mean(y_trues, dim=0, keepdim=True)
        ss_res = torch.sum((y_trues - y_hat_full) ** 2, dim=0)
        ss_tot = torch.sum((y_trues - y_mean) ** 2, dim=0)
        r2 = 1 - ss_res / (ss_tot + 1e-8)

        weights = self.criterion.weights.cpu()
        weighted_r2 = torch.sum(weights * r2)

        for i, name in enumerate(["DG", "DD", "DC", "GDM", "DT"]):
            self.log(f"val_r2_{name}", r2[i], prog_bar=True, on_epoch=True)
        self.log("val_weighted_r2", weighted_r2, prog_bar=True, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
        plateau = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode="min",
            factor=0.5,
            patience=2,
            threshold=0.001,
            min_lr=1e-7,
            verbose=False,
        )

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

In [7]:
# ======== Helpers ========
def tta_inference(model, images):
    preds = model(images)
    preds_lr = model(torch.flip(images, dims=[3]))
    preds_ud = model(torch.flip(images, dims=[2]))
    preds_lrud = model(torch.flip(images, dims=[2, 3]))
    preds_mean = (preds + preds_lr + preds_ud + preds_lrud) / 4.0
    return preds_mean

In [8]:
# Transform
img_size = 1000
infer_transform = T.Compose([
    T.Resize((img_size, img_size)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225])
])

# DataLoader
dataset = InferenceDataset(image_paths_df, transform=infer_transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results_dict = {}
for fold in range(5):
    model = MultiRegressionModel(model_name="efficientnet_b2", pretrained=False)
    model.load_state_dict(torch.load(f"/kaggle/input/csiro-naive-cnn-training/model_fold{fold}.pth"))
    model.eval()
    model.to(device)
    results = []
    with torch.no_grad():
        for batch in dataloader:
            images = batch
            images = images.to(device)
            preds = tta_inference(model, images)
            preds = preds.cpu().numpy()
            results.append(preds)
    results_dict[fold] = np.concatenate(results)

In [10]:
result_df = pd.DataFrame(np.mean([results_dict[fold] for fold in range(5)], axis=0), columns=["Dry_Green_g", "GDM_g", "Dry_Total_g"])
result_df["Dry_Dead_g"] = (result_df["Dry_Total_g"] - result_df["GDM_g"]).clip(lower=0)
result_df["Dry_Clover_g"] = (result_df["GDM_g"] - result_df["Dry_Green_g"]).clip(lower=0)
result_df['sample_id'] = image_paths_df['id']
result_df = pd.melt(result_df, id_vars='sample_id', value_vars=["Dry_Green_g", "Dry_Dead_g", "Dry_Clover_g", "GDM_g", "Dry_Total_g"], value_name='target')
result_df['sample_id'] = result_df['sample_id'] + '__' + result_df['variable']
result_df['target'] = result_df['target'].clip(0, 200)
result_df[['sample_id', 'target']].to_csv('submission.csv', index=False)